#### 数据整理 --- 百度数据集

In [1]:
import pandas as pd
import numpy as np
import math, os
from PIL import Image
import requests
from tqdm import tqdm
import hashlib
import requests
import json, time
import random
from IPython.display import display,HTML
from typing import  Optional, List
from io import BytesIO
import base64
from collections import defaultdict

import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False 
from glob import glob



In [2]:
def imgGallery(imglist: Optional[List], ids: Optional[List]=None, \
        scores: Optional[List] = None, tags: Optional[List]=None, \
        default_hight: Optional[str]='300px'):

    """ 显示图片：imglist：url列表或图片地址列表  """

    if ids is None:
        ids = []
    if tags is None:
        tags = []
    if scores is None:
        scores = []
    
    figures = []
    for i, img_path in enumerate(imglist):
        mid = ids[i] if ids else ''
        tag = tags[i] if tags else ""
        score = ', score=%.2f' % scores[i] if scores else ""
        file_info = f'<figcaption style="font-size: 1em;"><a target="_blank" href="{img_path}" style="display:inline;margin:1px"/> <br> {mid} {score} </figcaption>'
        desc_img = f'<figcaption style="font-size: 1em;"> <b> {tag} </b> <br> </figcaption>'
        figures.append(
            f"""
                <figure style="margin: 5px !important;">
                    {file_info}
                    <a target="_blank"  style="word-break: normal;"><img src="{img_path}" style="height: {default_hight}"></a>
                    {desc_img}
                </figure>
            """
            )

    
    return display(HTML(
                    data=f"""
                        <div style="display: flex; flex-flow: row wrap; text-align: center;">
                        {''.join(figures)}
                        </div>
                        """
                ))



In [9]:



def baidu_search(keyword, num_page=10, save_dir=os.getcwd()):
    """ 根据百度搜索词下载图片 """
    def get_img_url(url, url_list):
        """ 百度下载图片 """
        # headers = {
        # 'Cookie': 'BIDUPSID=FB2990F21BD9419435BFE31382227792; PSTM=1660009215; BAIDUID=FB2990F21BD94194567D64FA2BDB2217:FG=1; delPer=0; PSINO=5; BAIDUID_BFESS=FB2990F21BD94194567D64FA2BDB2217:FG=1; BA_HECTOR=0g8k0l0h0g8g0k80208hhsgl1hf6qoi17; ZFY=6nfp4tcgV3X15CkKZ2oPK8KtVgHDvLtL8FLNjir:A7UM:C; BDRCVFR[dG2JNJb_ajR]=mk3SLVN4HKm; BDRCVFR[-pGxjrCMryR]=mk3SLVN4HKm; BDRCVFR[tox4WRQ4-Km]=mk3SLVN4HKm; H_PS_PSSID=26350; BDORZ=FFFB88E999055A3F8A630C64834BD6D0; userFrom=www.baidu.com; ab_sr=1.0.1_ZmM1YzYyNDY1NTQ0ZDEyYzY0YjNmODYxYzFhM2ZjOGMyNzg0MjI0Yzk1NjM2Y2ViZDhkZWY0OGIyZTFmZTEyMmE2ZTg3Y2I1Mjk1MTRmOWRjNDhlMTZmMmMwNjBlYzUyZGY4YmIxODFkNjdkMDkxN2E5ZjRlZjFhOWEzN2RjY2UxZWUwZTYxM2M3MDQzZGU3YjBjODcwMGQ4ZTQ2MTlmYw==',
        # 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
        headers = {
            'Cookie': 'BIDUPSID=FB2990F21BD9419435BFE31382227792; PSTM=1660009215; BAIDUID=FB2990F21BD94194567D64FA2BDB2217:FG=1; BDORZ=FFFB88E999055A3F8A630C64834BD6D0; ai-studio-ticket=FC9652BB48144254ABE93F77DFA5811CCF59B7E30E364C54B77D2BAB875A85AA; H_PS_PSSID=26350; BAIDUID_BFESS=FB2990F21BD94194567D64FA2BDB2217:FG=1; delPer=0; PSINO=5; BDRCVFR[dG2JNJb_ajR]=mk3SLVN4HKm; BDRCVFR[-pGxjrCMryR]=mk3SLVN4HKm; BDRCVFR[R5kfAOC_u0m]=mk3SLVN4HKm; BA_HECTOR=8180a58k00010l25a4a55su11hf92mn16; ZFY=6nfp4tcgV3X15CkKZ2oPK8KtVgHDvLtL8FLNjir:A7UM:C; cleanHistoryStatus=0; userFrom=null; indexPageSugList=%5B%22%E7%99%BD%E8%89%B2%E6%B2%99%E5%8F%91%E7%9A%84%E5%AE%A2%E5%8E%85%22%2C%22%E6%9C%89%E7%99%BD%E8%89%B2%E6%B2%99%E5%8F%91%E7%9A%84%E5%AE%A2%E5%8E%85%22%2C%22%E5%AE%A2%E5%8E%85%E6%9D%A1%E7%BA%B9%E5%9C%B0%E6%AF%AF%22%2C%22%E5%AE%A2%E5%8E%85%E5%9C%B0%E6%AF%AF%20%E6%9D%A1%E7%BA%B9%22%2C%22%E5%AE%A2%E5%8E%85%E5%9C%B0%E6%AF%AF%22%2C%22%E7%BA%A2%E8%89%B2%E5%AE%A2%E5%8E%85%E5%9C%B0%E6%AF%AF%22%2C%22%E7%BA%A2%E8%89%B2%E5%9C%B0%E6%AF%AF%22%2C%22%E7%BA%A2%E8%89%B2%E8%89%B2%E5%9C%B0%E6%AF%AF%22%2C%22%E7%81%B0%E8%89%B2%E5%9C%B0%E6%AF%AF%22%5D; ab_sr=1.0.1_MWEyMDc0NTg2YjY2MmZlZjlkYWE5YzBjYzYzODZhODhiNDRlODk4OWIxMjY3ZDU2ODRmNjAyNDEyZjkzMThmMmFkNjc5MzdkZGQxMWY1NTdhNTMzZTYwNDMxNWNiNjFjZGRmYjVkYzA4M2I4NGRlYzFkYzRiNDRlMTRlODNiZTg3N2IxOWZkNDUwMDZ',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'
        }

        template = {}
        res = requests.get(url, headers=headers)
        bd_jdata = json.loads(res.text)
        for item in bd_jdata.get('data', []):
            img_dict={}
            if item:
                fromPageTitleEnc = item.get('fromPageTitleEnc')
                try:
                    replaceurl = item.get('replaceUrl')[1]
                    if replaceurl:
                        template[replaceurl.get('ObjURL')] = replaceurl.get('FromURL')
                        img_dict['url'] = replaceurl.get('ObjURL')
                        img_dict['tag'] = fromPageTitleEnc
                        url_list.append(img_dict)
                except:
                    replaceurl = item.get('replaceUrl')[0]
                    template[replaceurl.get('ObjURL')] = replaceurl.get('FromURL')
                    img_dict['url'] = replaceurl.get('ObjURL')
                    img_dict['tag'] = fromPageTitleEnc
                    url_list.append(img_dict)

    def get_ulrs(keyword, num_page):
        """ 获取下载链接及标签数据 """
        url_list = []
        for i in tqdm(range(1, num_page, 1)):
            base_url = f'https://image.baidu.com/search/acjson?tn=resultjson_com&logid=8513150795185198069&ipn=rj&ct=201326592&is=&fp=result&queryWord={keyword}&cl=2&lm=-1&ie=utf-8&oe=utf-8&adpicid=&st=-1&z=0&ic=undefined&hd=undefined&latest=undefined&copyright=undefined&word={keyword}&s=&se=&tab=&width=undefined&height=undefined&face=0&istype=2&qc=&nc=1&fr=&expermode=&force=&pn={i * 30}&rn=30&1603689505868='
            get_img_url(base_url, url_list)
            time_sleep = float(random.randint(0,9)/9) * 2
            time.sleep(time_sleep)

        return url_list

    def download(img_res, save_dir, keyword='白色窗帘'):
        """ 图像下载 """
        imgs_dwn = []
        headers1={ 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36' }
        for i, item in tqdm(enumerate(img_res)):
            url = item.get('url')
            tag = item.get('tag')
            time_sleep = float(random.randint(0,9)/9) * 2
            r = requests.get(url, headers=headers1, stream=True, timeout=30)
            time.sleep(time_sleep)

            if r.status_code==200:
                name=hashlib.md5(url.encode()).hexdigest() ## 编码图像
                save_path = os.path.join(save_dir, f"{name}.jpg")
                print(save_path)
                with open(save_path, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=32):
                        f.write(chunk)
                try:
                    _ = Image.open(save_path)
                    imgs_dwn.append({"imgfile": save_path, "tag": tag, 'keyword': keyword})
                except Exception as err:
                    print(f"img download error ：{save_path}")
                    os.remove(save_path)
                
                # if os.path.exists(save_path):
                #     imgs_dwn.append({"imgfile": save_path, "tag": tag, 'keyword': keyword})

        imgs_dwn = pd.DataFrame(imgs_dwn)
        return imgs_dwn
    
    urls_info = get_ulrs(keyword, num_page=num_page)
    # print(len(urls_info))
    if urls_info:
        print("图像下载 ... ")
        img_df = download(urls_info, save_dir, keyword)
        if len(img_df)>0:
            img_df.to_csv(os.path.join(save_dir, 'imgs.csv'), index=False)

    return urls_info



##### 根据搜索词下载图片

# keyword = '白色沙发的客厅'
save_dir = './baidu'
# urls_info= baidu_search(keyword, num_page=2, save_dir=save_dir)

num_page = 10

keyword_list = ['白色沙发的客厅', '白色的窗帘', '客厅的沙发']
for i, keyword in enumerate(keyword_list):
    save_path = os.path.join(save_dir, str(i))
    if not os.path.exists(save_path): os.mkdir(save_path)
    try:
        baidu_search(keyword, num_page, save_dir=save_path)
    except Exception as err:
        print(keyword, i, err)






100%|██████████| 1/1 [00:01<00:00,  1.28s/it]

30


In [4]:
### 合并搜索词下载结果

def get_csv(csv_path):
    img_dir = os.path.split(csv_path)[0]
    df1 = pd.read_csv(csv_path)
    df1['imgfile'] = df1['imgfile'].apply(lambda x: os.path.basename(str(x)))
    df1 = df1[df1['imgfile'].apply(lambda x: os.path.exists(os.path.join(img_dir, str(x))))]
    df1.reset_index(drop=True, inplace=True)
    return df1


df1 = get_csv('/data/1_qunosen/2022/res/CLIP/clip_chinese_text_encoder/cn_data_captions/baidu/1/imgs.csv')
df2 = get_csv('/data/1_qunosen/2022/res/CLIP/clip_chinese_text_encoder/cn_data_captions/baidu/2/imgs.csv')
df3 = get_csv('/data/1_qunosen/2022/res/CLIP/clip_chinese_text_encoder/cn_data_captions/baidu/3/imgs.csv')
df4 = get_csv('/data/1_qunosen/2022/res/CLIP/clip_chinese_text_encoder/cn_data_captions/baidu/4/imgs.csv')
df5 = get_csv('/data/1_qunosen/2022/res/CLIP/clip_chinese_text_encoder/cn_data_captions/baidu/5/imgs.csv')
df6 = get_csv('/data/1_qunosen/2022/res/CLIP/clip_chinese_text_encoder/cn_data_captions/baidu/6/imgs.csv')

In [5]:
df = pd.concat([df1,df2,df3,df4,df5,df6])
df

,imgfile,tag,keyword
0,94a8f1e5bb3933204eb241fcb3ff9dfe.jpg,红色落地窗帘,红色窗帘
1,6b5f0c5976555c6d0a931f9158e25ba0.jpg,"sun zero becca 节能金属扣眼窗帘 砖红色 40"" x 63"" 53386",红色窗帘
2,7d8468afce98b757bd30f37da30be52e.jpg,欧式窗帘红色,红色窗帘
3,04c3458b1fddcb0ff81a99b57a2223cf.jpg,红色喜庆蕾丝中式客厅窗帘效果图,红色窗帘
4,94c10548670db012e5a2267db6f34df0.jpg,ucharge 遮光窗帘适用于卧室和起居室隔热孔环顶部窗帘 1 个窗格 红色,红色窗帘
...,...,...,...
14,0f4935de3d147b8fac126bdc37c16c36.jpg,3米沙发客厅加工,客厅条纹地毯
15,faeb1100e2ae5a4091bad9ddd741cd9c.jpg,倍丽雅地毯 商用条纹地毯 客厅茶几办公用地毯 满铺方形地毯,客厅条纹地毯
16,e04c103f1130f283e9ce1883a826a4c0.jpg,卡通可爱简约条纹厨房洗手间客厅长条床边地毯进门垫浴室防滑地垫,客厅条纹地毯
17,feac2451fbca7c4931431590fe8bf2d2.jpg,现代简约客厅沙发黑色条纹地毯贴图,客厅条纹地毯


In [8]:
df['imgfile'] = df['imgfile'].apply(lambda x: os.path.join('./baidu/img', str(x)))
df


,imgfile,tag,keyword
0,/data/1_qunosen/2022/res/CLIP/clip_chinese_tex...,红色落地窗帘,红色窗帘
1,/data/1_qunosen/2022/res/CLIP/clip_chinese_tex...,"sun zero becca 节能金属扣眼窗帘 砖红色 40"" x 63"" 53386",红色窗帘
2,/data/1_qunosen/2022/res/CLIP/clip_chinese_tex...,欧式窗帘红色,红色窗帘
3,/data/1_qunosen/2022/res/CLIP/clip_chinese_tex...,红色喜庆蕾丝中式客厅窗帘效果图,红色窗帘
4,/data/1_qunosen/2022/res/CLIP/clip_chinese_tex...,ucharge 遮光窗帘适用于卧室和起居室隔热孔环顶部窗帘 1 个窗格 红色,红色窗帘
...,...,...,...
14,/data/1_qunosen/2022/res/CLIP/clip_chinese_tex...,3米沙发客厅加工,客厅条纹地毯
15,/data/1_qunosen/2022/res/CLIP/clip_chinese_tex...,倍丽雅地毯 商用条纹地毯 客厅茶几办公用地毯 满铺方形地毯,客厅条纹地毯
16,/data/1_qunosen/2022/res/CLIP/clip_chinese_tex...,卡通可爱简约条纹厨房洗手间客厅长条床边地毯进门垫浴室防滑地垫,客厅条纹地毯
17,/data/1_qunosen/2022/res/CLIP/clip_chinese_tex...,现代简约客厅沙发黑色条纹地毯贴图,客厅条纹地毯


In [10]:
df['keyword'].unique()

array(['红色窗帘', '白色窗帘', '枝形吊灯', '圆形吊灯', '红色色皮沙发', '客厅条纹地毯'], dtype=object)

In [18]:
kw = '枝形吊灯'

imgGallery(imglist=df[df['keyword']==kw]['imgfile'].tolist(), tags=df[df['keyword']==kw]['tag'].tolist(), ids= [os.path.basename(x) for x in df[df['keyword']==kw]['imgfile'].tolist()] )

